In [ ]:
!pip install \
   -U langchain-community \
   install pdfplumber \
   pinecone-client \
   langchain \
   openai \
   python-dotenv \
   pdfplumber \
   pinecone \
   tiktoken

  Using cached pinecone_client-4.1.2-py3-none-any.whl (216 kB)
  Using cached openai-1.35.10-py3-none-any.whl (328 kB)
  Attempting uninstall: pinecone-client
    Found existing installation: pinecone-client 3.2.2
    Uninstalling pinecone-client-3.2.2:
      Successfully uninstalled pinecone-client-3.2.2
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-pinecone 0.1.1 requires pinecone-client<4.0.0,>=3.2.2, but you have pinecone-client 4.1.2 which is incompatible.


In [ ]:
import openai
import langchain
import pinecone
import pdfplumber
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from dotenv import load_dotenv # Now you should be able to import dotenv
from pinecone import Pinecone, ServerlessSpec
from langchain.llms import OpenAI
from langchain.schema import Document
import tiktoken

In [ ]:
import os

In [ ]:
from google.colab import userdata

In [ ]:
openai_api=os.environ['OPENAI_API_KEY']= userdata.get('OPENAI_API_KEY')
pinecone_api=os.environ['PINECONE_API_KEY']= userdata.get('PINECONE_API_KEY')

In [ ]:
def read_doc(directory):
    documents = []
    for filename in os.listdir(directory):
        if filename.endswith('.pdf'):
            with pdfplumber.open(os.path.join(directory, filename)) as pdf:
                for page in pdf.pages:
                    documents.append(Document(page_content=page.extract_text(), metadata={'source': filename}))
    return documents

In [ ]:
doc=read_doc('/content/')
print(len(doc))


58


In [ ]:
doc

[Document(metadata={'source': 'budget_speech.pdf'}, page_content='GOVERNMENT OF INDIA\nBUDGET 2023-2024\nSPEECH\nOF\nNIRMALA SITHARAMAN\nMINISTER OF FINANCE\nFebruary 1, 2023'),
 Document(metadata={'source': 'budget_speech.pdf'}, page_content=''),
 Document(metadata={'source': 'budget_speech.pdf'}, page_content='CONTENTS\nPART-A\nPage No.\n\uf0b7 Introduction 1\n\uf0b7 Achievements since 2014: Leaving no one behind 2\n\uf0b7 Vision for Amrit Kaal – an empowered and inclusive economy 3\n\uf0b7 Priorities of this Budget 5\ni. Inclusive Development\nii. Reaching the Last Mile\niii. Infrastructure and Investment\niv. Unleashing the Potential\nv. Green Growth\nvi. Youth Power\nvii. Financial Sector\n\uf0b7 Fiscal Management 24\nPART B\nIndirect Taxes 27\n\uf0b7 Green Mobility\n\uf0b7 Electronics\n\uf0b7 Electrical\n\uf0b7 Chemicals and Petrochemicals\n\uf0b7 Marine products\n\uf0b7 Lab Grown Diamonds\n\uf0b7 Precious Metals\n\uf0b7 Metals\n\uf0b7 Compounded Rubber\n\uf0b7 Cigarettes\nDirect

In [ ]:
def chunk_data(docs, chunk_size=800, chunk_overlap=50):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  doc=text_splitter.split_documents(docs)
  return docs

In [ ]:
documents=chunk_data(docs=doc)
len(documents)

58

In [ ]:
documents[2]

Document(metadata={'source': 'budget_speech.pdf'}, page_content='CONTENTS\nPART-A\nPage No.\n\uf0b7 Introduction 1\n\uf0b7 Achievements since 2014: Leaving no one behind 2\n\uf0b7 Vision for Amrit Kaal – an empowered and inclusive economy 3\n\uf0b7 Priorities of this Budget 5\ni. Inclusive Development\nii. Reaching the Last Mile\niii. Infrastructure and Investment\niv. Unleashing the Potential\nv. Green Growth\nvi. Youth Power\nvii. Financial Sector\n\uf0b7 Fiscal Management 24\nPART B\nIndirect Taxes 27\n\uf0b7 Green Mobility\n\uf0b7 Electronics\n\uf0b7 Electrical\n\uf0b7 Chemicals and Petrochemicals\n\uf0b7 Marine products\n\uf0b7 Lab Grown Diamonds\n\uf0b7 Precious Metals\n\uf0b7 Metals\n\uf0b7 Compounded Rubber\n\uf0b7 Cigarettes\nDirect Taxes 30\n\uf0b7 MSMEs and Professionals\n\uf0b7 Cooperation\n\uf0b7 Start-Ups\n\uf0b7 Appeals\n\uf0b7 Better targeting of tax concessions\n\uf0b7 Rationalisation\n\uf0b7 Others\n\uf0b7 Personal Income Tax\nAnnexures 35\n\uf0b7 Annexure to Part B o

In [ ]:
load_dotenv('.env')

False

In [ ]:

embeddings=OpenAIEmbeddings(api_key=openai_api)
embeddings

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7ea42e6d8310>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7ea42e6dafb0>, model='text-embedding-ada-002', deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key='sk-proj-9AM1WuYK1KZ4mjnOH4fhT3BlbkFJVvWveetFVaOOb7s3bDSx', openai_organization=None, allowed_special=set(), disallowed_special='all', chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None)

In [ ]:
vectors=embeddings.embed_query("How are you?")
len(vectors)

1536

In [ ]:
vectors

[-0.016785908412158042,
 -0.012151270116836888,
 0.006627965687606043,
 -0.026018159342696565,
 -0.01616878054948239,
 0.01762520513661754,
 -0.011114493891931487,
 -0.0099234347700346,
 -0.018131250431046412,
 -0.010417137246439636,
 0.0278695466560138,
 0.0016508201293049156,
 -0.00733766375413787,
 -0.011651395765758651,
 0.007238923072592348,
 -0.015391197915142053,
 0.028363250063741408,
 -0.011830363367475229,
 0.013959458032731997,
 -0.0205997656070393,
 0.00252868606461311,
 0.006344086460993312,
 0.0009997490514024384,
 -0.008263357876457773,
 -0.01588490039154709,
 -0.007794339173455263,
 0.025117151322085612,
 -0.012404292764051323,
 0.02230304282936083,
 -0.02515417884483454,
 0.005609702758413819,
 0.0076955989575710265,
 -0.013169533511690397,
 0.004014424029647617,
 0.008757060352862809,
 -0.022290699080014423,
 0.004020595438659533,
 -0.01043565193913667,
 0.0203282291984504,
 -0.006337915051981395,
 0.0270302499315543,
 0.001255857519538152,
 -0.005239425202618115,
 -0

In [ ]:
use_serverless = True

In [ ]:
!pip install langchain_pinecone

  Using cached pinecone_client-3.2.2-py3-none-any.whl (215 kB)
  Attempting uninstall: pinecone-client
    Found existing installation: pinecone-client 4.1.2
    Uninstalling pinecone-client-4.1.2:
      Successfully uninstalled pinecone-client-4.1.2


In [ ]:
from langchain.vectorstores import Pinecone
from langchain_pinecone import PineconeVectorStore

In [ ]:
pinecone_host='https://langchainvector-xot7p5a.svc.aped-4627-b74a.pinecone.io'

index_name="pinecone-chatbot"
index = pinecone.Index(index_name,host=pinecone_host)

In [ ]:
vectorstore=PineconeVectorStore.from_documents(doc, embeddings, index_name=index_name)

In [ ]:
from pinecone import Pinecone

pc = Pinecone(api_key=pinecone_api)
index = pc.Index("pinecone-chatbot")

In [ ]:
vectors = [
    {"id": str(doc.id), "values": [float(x) for x in embeddings.embed_query(doc.page_content)], "metadata": doc.metadata}
    for doc in doc
]
index.upsert(vectors=vectors, index_name=index_name)

{'upserted_count': 58}

In [ ]:
def retrieve_query(query,k=2):
    query_vector = embeddings.embed_query(query)
    matching_results = index.query(vector=query_vector, top_k=k, include_metadata=True)
    return matching_results

In [99]:
!pip install -U langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 1.6 MB/s eta 0:00:00


In [100]:

from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI
from langchain_openai import ChatOpenAI

In [101]:
llm=ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0.5)
chain=load_qa_chain(llm,chain_type="stuff")

In [104]:
def retrieve_answers(query):
    doc_search = retrieve_query(query)
    print(doc_search)
    response = chain.run(input_documents=doc_search, question=query)
    return response

In [105]:
query = "what has National Rural Livelihood Mission has achived?"
answer = retrieve_answers(query)
print(answer)

[Document(metadata={'source': 'budget_speech.pdf', 'text': '4\n1) Economic Empowerment of Women: Deendayal Antyodaya Yojana\nNational Rural Livelihood Mission has achieved remarkable success\nby mobilizing rural women into 81 lakh Self Help Groups. We will\nenable these groups to reach the next stage of economic\nempowerment through formation of large producer enterprises or\ncollectives with each having several thousand members and\nmanaged professionally. They will be helped with supply of raw\nmaterials and for better design, quality, branding and marketing of\ntheir products. Through supporting policies, they will be enabled to\nscale up their operations to serve the large consumer markets, as\nhas been the case with several start-ups growing into ‘Unicorns’.\n2) PM VIshwakarma KAushal Samman (PM VIKAS): For centuries,\ntraditional artisans and craftspeople, who work with their hands\nusing tools, have brought renown for India. They are generally\nreferred to as Vishwakarma. The ar

In [106]:
query = "How much the agriculture target will be increased by how many crore?"
answer = retrieve_answers(query)
print(answer)

[Document(metadata={'source': 'budget_speech.pdf', 'text': "7\nfarmers in contributing to the health of fellow citizens by growing these\n‘Shree Anna’.\n22. Now to make India a global hub for 'Shree Anna', the Indian Institute\nof Millet Research, Hyderabad will be supported as the Centre of Excellence\nfor sharing best practices, research and technologies at the international\nlevel.\nAgriculture Credit\n23. The agriculture credit target will be increased\nto ` 20 lakh crore with focus on animal husbandry, dairy and fisheries.\nFisheries\n24. We will launch a new sub-scheme of PM Matsya Sampada Yojana\nwith targeted investment of ` 6,000 crore to further enable activities of\nfishermen, fish vendors, and micro & small enterprises, improve value chain\nefficiencies, and expand the market.\nCooperation\n25. For farmers, especially small and marginal farmers, and other\nmarginalised sections, the government is promoting cooperative-based\neconomic development model. A new Ministry of Coo